# Task 1 – Data Cleaning & Formatting

**Objective:**  
To clean and format the client-provided survey dataset into an SPSS-ready `.sav` file for Q import.

**Tools Used:**  
1. **Python** (pandas, numpy, pyreadstat,re,os,datetime)
2. **Excel** (for data validation)

**Summraised Logical Notes (Errors might be found):**
1. **Check screening variables** – Gender, Age, Postcode, or CompletedDate not provided.
2. **Age under 18 respondents** – "Under 18" should be terminated.  
3. **Brand awareness conflict** Selected "None of these"(Q1_99) but still answered Q2–Q5.  
4. **Brand usage conflict** Not “Origin” users (Q2 ≠ 4) but still answered Q3–Q5.  
5. **Advertising conflict** Said *“No”* or *“Don’t know”* in Q6 but still answered Q7.
6. **Inconsistent brand awareness** If selected “None of these” (Q1_99) ,then should not selected other brands in Q1 .


**Overview of Steps**
1. Constructing Input and Output Paths 
2. Read raw data 
3. Clean column names and remove leading/trailing spaces and replace non-standard quotes 
4. Data cleaning and format conversion 
5. Export to SPSS file (.sav)
6. Display the ratio of Origin users vs. non-Origin users
7. Display inspection results for all text columns
8. Save processed Excel file for manual inspection

## Step 0 - Import Core Libraries

Before running any cleaning functions, the core Python libraries used for data manipulation and export are imported.

- **pandas** – to handle and transform tabular data efficiently  
- **numpy** – for numerical operations and managing missing values (`NaN`)  
- **datetime** – to process date and time information (used for creating the “Wave” variable)  
- **pyreadstat** – to export the cleaned dataset to SPSS `.sav` format  
- **re** – for regular expressions used in text and pattern matching  
- **os** – for managing file paths when reading and saving outputs


In [33]:
import pandas as pd
import numpy as np
from datetime import datetime
import pyreadstat
import re
import os

## Step 1 – Constructing Input and Output Paths

This ensures the codes can automatically locate the raw data and save all cleaned outputs to the correct folder.

- **BASE_DIR**   – the current working directory of this Notebook  
- **input_path**   – points to the raw dataset stored one level above  
- **output_sav_path**   – location where the cleaned SPSS `.sav` file will be exported  
- **output_check_path**   – location for the Excel output used for manual verification


In [34]:
BASE_DIR = os.getcwd()  # Get current working directory
print("BASE_DIR:", BASE_DIR)

input_path = os.path.join(BASE_DIR, "..", "EXAMPLE DATA FILE.xlsx")         # Path to the raw data file (one level above)
output_sav_path = os.path.join(BASE_DIR, "cleaned_data.sav")                # Output path for the *.sav file
output_check_path = os.path.join(BASE_DIR, "cleaned_data_check.xlsx")       # Output path for the Excel check file

print("📂 Input file path:", input_path)
print("💾 Output SAV path:", output_sav_path)
print("💾 Output Excel check path:", output_check_path)

BASE_DIR: /Users/sunfangyu/Desktop/Charles Data Task/Task1
📂 Input file path: /Users/sunfangyu/Desktop/Charles Data Task/Task1/../EXAMPLE DATA FILE.xlsx
💾 Output SAV path: /Users/sunfangyu/Desktop/Charles Data Task/Task1/cleaned_data.sav
💾 Output Excel check path: /Users/sunfangyu/Desktop/Charles Data Task/Task1/cleaned_data_check.xlsx


## Step 2 – Read Raw Data

Load the original dataset to verify that the file path is correct  
and to inspect the raw structure and variable names before cleaning.


In [35]:
# Read raw data
df = pd.read_excel(input_path)
df

,ID,What is your gender?,What is your age?,What is your postcode?,Which of the following brands of electricity providers are you aware of?,Which of the following brands of electricity providers are you aware of? (Other (please specify)),And which ONE of these brands is your main provider?,And which ONE of these brands is your main provider? (Other (please specify)),"Thinking about ‘Origin’, how favourable is your overall impression of them?",How likely are you to recommend ‘Origin’ to friends or family?,...,How would you rate ‘Origin’ on each of the following? (Innovation),"In the past 12 months, have you seen or heard any advertising for ‘Origin’?",Where did you see or hear advertising for ‘Origin’?,Where did you see or hear advertising for ‘Origin’? (Other (please specify)),Which of the following best describes your current work status?,Which of the following best describes your current work status? (Other (please specify)),Which of the following best describes your total annual household income?,Which of the following best describes your household structure?,Which of the following best describes your household structure? (Other (please specify)),CompletedDate
0,76,Female,65+,6128,Synergy; AGL; Origin; Red Energy,,Origin,,Very favourable,1,...,Fair,Don't know,NaN,NaN,Retired,,"$60,000–$89,999",Group household / share house,,2025-08-04
1,78,Female,65+,6002,None of these,,Origin,,Very unfavourable,7,...,Very poor,No,NaN,NaN,Student,,"$60,000–$89,999",Single parent with children at home,,2025-08-04
2,79,Female,35-44,6289,Synergy; Western Power; Origin; Horizon Power;...,ATCO,Red Energy,,Very favourable,6,...,Excellent,No,NaN,NaN,Unemployed and looking for work,,"$60,000–$89,999","Couple, no children",,2025-08-04
3,82,Female,55-64,6122,AGL; Origin; Horizon Power,,Origin,,Neutral,2,...,Fair,No,NaN,NaN,Retired,,"$90,000–$119,999","Couple, no children",,2025-08-04
4,85,Female,25-34,6162,None of these,,Origin,,Very favourable,6,...,Fair,Yes,TV; Online / Social media; Outdoor (billboards...,NaN,Working full time,,"Less than $30,000","Single, no children",,2025-08-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
198,578,Male,45-54,6123,Western Power; AGL; Origin; Horizon Power,,Origin,,Very unfavourable,3,...,Very poor,No,NaN,NaN,Working part time,,"Less than $30,000","Single, no children",,2025-08-30
199,580,Male,18-24,6385,Synergy; Western Power; Red Energy; Other (ple...,Infinite energy,Other (please specify),Infinite energy,,,...,,Don't know,NaN,NaN,Retired,,"Less than $30,000","Couple, no children",,2025-08-30
200,157,Female,35-44,6376,None of these,,,,,,...,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT
201,321,Male,35-44,6150,Synergy; Western Power; Origin; Horizon Power;...,,Synergy,,,,...,,Yes,"Radio; Print (newspaper, magazine)",NaN,NaN,NaN,NaN,NaN,NaN,NaT


## Step 3 – Clean Column Names

Before running any cleaning functions, column names are standardized.  
This step removes leading and trailing spaces and replaces non-standard quotation marks  
to ensure consistency and prevent errors when referencing column names in later steps.


In [36]:

df.columns = df.columns.str.strip().str.replace('‘', "'").str.replace('’', "'")


## Step 4 – Data Cleaning and Format Conversion

This step performs the full cleaning pipeline by calling the main function `clean_and_format_data(df)`.

The function automates several processes:
1. Removing invalid or inconsistent cases  
2. Processing multi-response questions (one-hot encoding)  
3. Converting text labels to numeric codes according to the questionnaire
4. Renaming and reordering columns according to the questionnaire  
5. Creating the weekly “Wave” variable  
6. Generating SPSS variable and value labels  


### Step 4.1 – Remove Invalid and Inconsistent Records

This sub-step identifies and removes all invalid or logically inconsistent responses.  
Each type of error is checked and documented individually to maintain transparency in the cleaning process.

**Errors handled in this step:**
1. **Error 1 – Missing demographic data**  
   Records missing key variables: *Gender, Age, Postcode,* or *CompletedDate.*

2. **Error 2 – Underage respondents**  
   Participants under 18 years old who should have been terminated.

3. **Error 3 – Brand awareness conflict**  
   Selected *“None of these”* (Q1_99) but still answered Q2–Q5 (Brand related questions).

4. **Error 4 – Brand usage conflict**  
   Did not select *“Origin”* as main provider (Q2 ≠ 4) but still answered Q3–Q5 (Origin related quetsiones).

5. **Error 5 – Advertising awareness conflict**  
   Said *“No”* or *“Don’t know”* in Q6 but still answered Q7 (Origin advertising related questions).

6. **Error 6 - Inconsistent brand awareness** 
   If selected “None of these” (Q1_99) ,then should not selected other brands in Q1 .


For each error type, the number of affected records and their IDs are printed for reference.  
After this step, only valid and logically consistent records remain in the dataset.


In [37]:

"""
Remove invalid records – delete those with logic errors and missing data
"""

# Record original row count
original_count = len(df)

# Remove completely blank rows
df = df.dropna(how='all').reset_index(drop=True)

# Handle spaces and empty strings: convert whitespace-only values to NaN
df = df.replace(r'^\s*$', np.nan, regex=True)

# Key variables: Gender, Age, Postcode, CompletedDate
key_columns = ['What is your gender?', 'What is your age?', 'What is your postcode?', 'CompletedDate']

# Error1 – Missing key variables
# Remove rows with missing key variables
error1 = df[key_columns].isna().any(axis=1)
print(f"Error1: Number of records with missing key variables: {error1.sum()}")

# Display IDs of these records for inspection
if error1.sum() > 0:
    error_ids = df.loc[error1, 'ID'].tolist()
    print(f"Error1 ID: {error_ids}")
    
df = df[~error1].reset_index(drop=True)

# Error2
# Remove rows where age is "Under 18" (should terminate based on survey logic)
error2 = df['What is your age?'] == 'Under 18'
print(f"Error2: Number of records where age is under 18:{error2.sum()}")
df = df[~error2].reset_index(drop=True)

# Display IDs of these records for inspection
if error2.sum() > 0:
    error_ids = df.loc[error2, 'ID'].tolist()
    print(f"Error2 ID: {error_ids}")

# Error6: Q1 contains "None of these"(Q1_99) but also includes other brands
if 'Which of the following brands of electricity providers are you aware of?' in df.columns:
    q1_col = 'Which of the following brands of electricity providers are you aware of?'

    # Normalize delimiters for accurate text matching
    df[q1_col] = df[q1_col].astype(str)
    df[q1_col] = df[q1_col].str.replace(',', ';', regex=False)
    df[q1_col] = df[q1_col].str.replace('；', ';', regex=False)
    df[q1_col] = df[q1_col].str.replace(r'\s*;\s*', ';', regex=True)

    # List all brand names (excluding "None of these")
    brand_names = ['Synergy', 'Western Power', 'AGL', 'Origin', 'Horizon Power', 'Red Energy']

    # Detect responses that contain "None of these" plus any other brand
    contains_none = df[q1_col].str.contains('None of these', case=False, na=False)
    contains_other = df[q1_col].str.contains('|'.join([re.escape(b) for b in brand_names]), case=False, na=False)
    error6 = contains_none & contains_other

    print(f"Error6: Number of records with 'None of these' plus other Q1 options: {error6.sum()} records")
    if error6.sum() > 0:
        error_ids = df.loc[error6, 'ID'].tolist()
        print(f"Error6 ID: {error_ids}")

    df = df.loc[~error6].reset_index(drop=True) 

# Error 3: 
# Select participants who chose Q1_99 "None of these" (SKIP to Q6).
q1_none_selected = df['Which of the following brands of electricity providers are you aware of?'].str.contains(
    'None of these', na=False, regex=False)

# For records selecting Q1_99 ("None of these"), remove Q2–Q5 data.
q2_q5_columns = [
    'And which ONE of these brands is your main provider?',
    'Thinking about \'Origin\', how favourable is your overall impression of them?',
    'How likely are you to recommend \'Origin\' to friends or family?',
    'How would you rate \'Origin\' on each of the following? (Trustworthiness)',
    'How would you rate \'Origin\' on each of the following? (Value for money)',
    'How would you rate \'Origin\' on each of the following? (Customer service)',
    'How would you rate \'Origin\' on each of the following? (Innovation)'
]

# Only keep the columns exist
existing_q2_q5_columns = [col for col in q2_q5_columns if col in df.columns]

# Check logic Error3: selected Q1_99, but answered Q2-Q5.
if existing_q2_q5_columns:
    error3 = q1_none_selected & df[existing_q2_q5_columns].notna().any(axis=1)
    print(f"Error3: Number of selected Q1_99,but answered Q2-Q5: {error3.sum()}records")

    # Show Error3 ID for checking to delete
    if error3.sum() > 0:
        error_ids = df.loc[error3, 'ID'].tolist()
        print(f"Error3 ID: {error_ids}")

    df = df.loc[~error3].reset_index(drop=True)

# Error4
# Check logic error4：Not Origin user,but answered Q3-Q5
if 'And which ONE of these brands is your main provider?' in df.columns:
    origin_main_provider = df['And which ONE of these brands is your main provider?'] == 'Origin'

    # Check if Q3-Q5 have data
    q3_q5_columns = [
        'Thinking about \'Origin\', how favourable is your overall impression of them?',
        'How likely are you to recommend \'Origin\' to friends or family?',
        'How would you rate \'Origin\' on each of the following? (Trustworthiness)',
        'How would you rate \'Origin\' on each of the following? (Value for money)',
        'How would you rate \'Origin\' on each of the following? (Customer service)',
        'How would you rate \'Origin\' on each of the following? (Innovation)'
    ]

    # Only keep the existed actual column
    existing_q3_q5_columns = [col for col in q3_q5_columns if col in df.columns]

    if existing_q3_q5_columns:
        # Consider answered if any of Q3–Q5 columns contain data
        has_q3_q5_data = df[existing_q3_q5_columns].notna().any(axis=1)

        # Non-Origin users who answered Q3–Q5 and did not select Q1_99
        error4 = ~origin_main_provider & has_q3_q5_data & ~q1_none_selected
        print(f"Error4: Number of (non-Origin users answered Q3–Q5)records: {error4.sum()}records")

        # Display IDs of these records for inspection
        if error4.sum() > 0:
            error_ids = df.loc[error4, 'ID'].tolist()
            print(f"IDs of error4 records: {error_ids}")

        # Display IDs of these records for inspection
        if error4.sum() > 0:
            error_ids = df.loc[error4, 'ID'].tolist()
            print(f"Error4 ID: {error_ids}")

        # Use .loc to avoid warnings
        df = df.loc[~error4].reset_index(drop=True)

#Error5
# Check logic error5：Q6 answered "No" or "Don't know" but also answered Q7
if 'In the past 12 months, have you seen or heard any advertising for \'Origin\'?' in df.columns:
    q6_no_advertising = df['In the past 12 months, have you seen or heard any advertising for \'Origin\'?'].isin(
        ['No', 'Don\'t know'])
    has_q7_data = df['Where did you see or hear advertising for \'Origin\'?'].notna()

    error5 = q6_no_advertising & has_q7_data
    print(f"Error5: Number of (Q6 answered \"No\" or \"Don't know\" but also answered Q7)records: {error5.sum()}records")
    # Use .loc to avoid warnings
    df = df.loc[~error5].reset_index(drop=True)

    # Display IDs of these records for inspection
    if error5.sum() > 0:
        error_ids = df.loc[error5, 'ID'].tolist()
        print(f"Error5 ID: {error_ids}")

final_count = len(df)
print(f"Invalid record removal completed: {original_count} -> {final_count}")



Error1: Number of records with missing key variables: 3
Error1 ID: [157, 321, 560]
Error2: Number of records where age is under 18:3
Error2 ID: [189, 487, 577]
Error6: Number of records with 'None of these' plus other Q1 options: 1 records
Error6 ID: [91]
Error3: Number of selected Q1_99,but answered Q2-Q5: 3records
Error3 ID: [78, 85, 96]
Error4: Number of (non-Origin users answered Q3–Q5)records: 0records
Error5: Number of (Q6 answered "No" or "Don't know" but also answered Q7)records: 0records
Invalid record removal completed: 203 -> 193


### Step 4.2 – Process Multi-Response Questions (One-Hot Encoding)

This step converts multi-response questions into analyzable binary variables. 

Questions such as **Q1 (Brand Awareness)** and **Q7 (Advertising Channels)** allow respondents to select multiple options.  
Each option is split into a separate column indicating selection (1 = Selected, 0 = Not selected).

These transformations make it possible to calculate awareness or channel usage percentages for each option consistently.

#### Step 4.2a – Function: create_multiresponse_columns()

The helper function `create_multiresponse_columns()` automates the one-hot encoding process for multi-response questions.

**How it works**
- Creates a new binary column for every response option  
- Handles **“Other (please specify)”** responses by checking if text exists in the related “Other” column  
- Handles **“None of these”** as an exclusive option  
- Uses `str.contains()` to detect whether each label appears in the original response string

This ensures consistent, automated encoding for all multi-response questions (e.g., Q1 and Q7).

In [38]:
def create_multiresponse_columns(df, column, options, question_code):
    """
    Create one-hot encoded columns for multi-response questions
    """
    # Create new binary columns
    for label, code in options.items():
        new_col_name = f"{question_code}_{code}"

        if label == 'Other (please specify)':
            # "Others" option requires checking a separate column
            other_col = f"{column} (Other (please specify))"
            if other_col in df.columns:
                df[new_col_name] = df[other_col].notna().astype(int)
            else:
                df[new_col_name] = 0
        elif label == 'None of these':
            # Check if "None of these" is included (use regex for matching
            df[new_col_name] = df[column].str.contains(re.escape('None of these'), na=False).astype(int)
        else:
            # Check if the brand is included (use regex for matching)
            df[new_col_name] = df[column].str.contains(re.escape(label), na=False, regex=False).astype(int)

    return df

#### Step 4.2b – Apply One-Hot Encoding to Q1 and Q7

After defining the helper function, the next step is to apply it to the relevant multi-response questions:

- **Q1 (Brand Awareness):** converts each electricity provider into binary columns (`Q1_1`–`Q1_99`).  
- **Q7 (Advertising Channels):** converts each media channel into binary columns (`Q7_1`–`Q7_97`).

Each option is coded as:
- **1** → Selected  
- **0** → Not selected



In [39]:

"""
Process one-hot encoding for multi-response questions
"""

# Q1: Bradn Awarness
q1_options = {
    'Synergy': '1',
    'Western Power': '2',
    'AGL': '3',
    'Origin': '4',
    'Horizon Power': '5',
    'Red Energy': '6',
    'Other (please specify)': '97',
    'None of these': '99'
}

# Process Q1 multi-response
q1_col = 'Which of the following brands of electricity providers are you aware of?'
df = create_multiresponse_columns(df, q1_col, q1_options, 'Q1')

# Q7: Advertising channels
q7_options = {
    'TV': '1',
    'Online / Social media': '2',
    'Outdoor (billboards, bus stops, etc.)': '3',
    'Radio': '4',
    'Print (newspaper, magazine)': '5',
    'Other (please specify)': '97'
}

# Process Q7 multi-response
q7_col = 'Where did you see or hear advertising for \'Origin\'?'
df = create_multiresponse_columns(df, q7_col, q7_options, 'Q7')




#### Step 4.3 – Mapping Logic Overview (Aligned with QNA)

All codes below follow the official Example QNA codeframe.

| Variable | Code Mapping |
|-----------|-----------------------------|
| **S1 – Gender** | 1 = Male    2 = Female    3 = Non-binary / Other    99 = Prefer not to say |
| **S2 – Age** | 1 = Under 18    2 = 18–24    3 = 25–34    4 = 35–44    5 = 45–54    6 = 55–64    7 = 65+ |
| **S3 – Postcode** | Open numeric input (4-digit postcode) |
| **Q2 – Main Provider** | 1 = Synergy    2 = Western Power    3 = AGL    4 = Origin    5 = Horizon Power    6 = Red Energy    97 = Other    99 = None of these |
| **Q3 – Brand Favourability** | 1 = Very unfavourable    2 = Somewhat unfavourable    3 = Neutral    4 = Somewhat favourable    5 = Very favourable |
| **Q4a – Likelihood to Recommend** | 0 = Not at all likely    1–9 = Scale points    10 = Extremely likely |
| **Q5 – Brand Values (MR scale)** | Attribute codes 1–4 = Trustworthiness / Value for money / Customer service / Innovation    Each attribute rating: 1 = Very poor    2 = Poor    3 = Fair    4 = Good    5 = Excellent    98 = Don’t know |
| **Q6 – Ad Awareness** | 1 = Yes    2 = No    98 = Don’t know |
| **D1 – Work Status** | 1 = Working full time    2 = Working part time    3 = Self-employed    4 = Student    5 = Unemployed and looking for work    6 = Retired    97 = Other (please specify) |
| **D2 – Household Income** | 1 = Less than $30 000    2 = $30 000–$59 999    3 = $60 000–$89 999    4 = $90 000–$119 999    5 = $120 000–$149 999    6 = $150 000 or more    99 = Prefer not to say |
| **D3 – Household Structure** | 1 = Live alone    2 = Single, no children    3 = Single parent with children at home    4 = Couple, no children    5 = Couple with children at home    6 = Group household / share house    97 = Other (please specify) |


In [40]:

"""
Convert text labels to numeric codes
"""

# S1: Gender
gender_mapping = {
    'Male': 1,
    'Female': 2,
    'Non-binary / Other': 3,
    'Prefer not to say': 99
}
df['What is your gender?'] = df['What is your gender?'].map(gender_mapping)

# S2: Age
age_mapping = {
    '18-24': 2,
    '25-34': 3,
    '35-44': 4,
    '45-54': 5,
    '55-64': 6,
    '65+': 7
}
df['What is your age?'] = df['What is your age?'].map(age_mapping)

# Q2: Main Supplier
provider_mapping = {
    'Synergy': 1,
    'Western Power': 2,
    'AGL': 3,
    'Origin': 4,
    'Horizon Power': 5,
    'Red Energy': 6,
    'Other (please specify)': 97,
    'None of these': 99
}
df['And which ONE of these brands is your main provider?'] = df[
    'And which ONE of these brands is your main provider?'].map(provider_mapping)

# Q3: Brand perception (only for Origin users)
favourable_mapping = {
    'Very unfavourable': 1,
    'Somewhat unfavourable': 2,
    'Neutral': 3,
    'Somewhat favourable': 4,
    'Very favourable': 5
}
df['Thinking about \'Origin\', how favourable is your overall impression of them?'] = df[
    'Thinking about \'Origin\', how favourable is your overall impression of them?'].map(favourable_mapping)

# Q4a: Likelihood to recommend
def convert_recommendation(value):
    if pd.isna(value):
        return np.nan
    if 'Not at all likely' in str(value) or value == '0':
        return 0
    if 'Extremely likely' in str(value) or value == '10':
        return 10
    try:
        return int(float(value))
    except:
        return np.nan

df['How likely are you to recommend \'Origin\' to friends or family?'] = df[
    'How likely are you to recommend \'Origin\' to friends or family?'].apply(convert_recommendation)

# Q5: Attribute ratings
rating_mapping = {
    'Very poor': 1,
    'Poor': 2,
    'Fair': 3,
    'Good': 4,
    'Excellent': 5,
    'Don\'t know': 98
}

rating_columns = [
    'How would you rate \'Origin\' on each of the following? (Trustworthiness)',
    'How would you rate \'Origin\' on each of the following? (Value for money)',
    'How would you rate \'Origin\' on each of the following? (Customer service)',
    'How would you rate \'Origin\' on each of the following? (Innovation)'
]

for col in rating_columns:
    df[col] = df[col].map(rating_mapping)

# Q6: Advertising exposure
advertising_mapping = {
    'Yes': 1,
    'No': 2,
    'Don\'t know': 98
}
df['In the past 12 months, have you seen or heard any advertising for \'Origin\'?'] = df[
    'In the past 12 months, have you seen or heard any advertising for \'Origin\'?'].map(advertising_mapping)

# D1: Work status
work_mapping = {
    'Working full time': 1,
    'Working part time': 2,
    'Self-employed': 3,
    'Student': 4,
    'Unemployed and looking for work': 5,
    'Retired': 6,
    'Other (please specify)': 97
}
df['Which of the following best describes your current work status?'] = df[
    'Which of the following best describes your current work status?'].map(work_mapping)

# D2: Income
income_mapping = {
    'Less than $30,000': 1,
    '$30,000-$59,999': 2,
    '$60,000–$89,999': 3,
    '$90,000–$119,999': 4,
    '$120,000–$149,999': 5,
    '$150,000 or more': 6,
    'Prefer not to say': 99
}
df['Which of the following best describes your total annual household income?'] = df[
    'Which of the following best describes your total annual household income?'].map(income_mapping)

# D3: Household structure
household_mapping = {
    'Live alone': 1,
    'Single, no children': 2,
    'Single parent with children at home': 3,
    'Couple, no children': 4,
    'Couple, with children at home': 5,
    'Group household / share house': 6,
    'Other (please specify)': 97
}
df['Which of the following best describes your household structure?'] = df[
    'Which of the following best describes your household structure?'].map(household_mapping)





### Step 4.4 – Rename and Reorder Columns

After all variable transformations are complete, this step standardises the column names and re-orders them  
to match the official questionnaire structure.  

**Main actions**
1. **Rename columns** – Replace long question texts with concise variable names (e.g.,  
   `What is your gender?` → `S1`, `How would you rate 'Origin'... (Customer service)` → `Q5_3`).  
   This follows the naming convention used in the QNA for SPSS/Q import.  
2. **Add special mappings** – Detect and rename “Other (please specify)” fields  
   (e.g., `Q1_97_Oth`, `Q2_97_Oth`, `D1_97_Oth`).  
3. **Reorder columns** – Arrange variables in logical survey order:  
   Screening (S-series) → Brand (Q-series) → Demographics (D-series) → Derived (Wave).  
4. **Drop redundant raw columns** – Remove the original multi-response text fields  
   now replaced by one-hot encoded variables.?


In [41]:

"""
Rename columns and adjust their order
"""

# Rename columns
column_mapping = {
    'ID': 'ID',
    'What is your gender?': 'S1',
    'What is your age?': 'S2',
    'What is your postcode?': 'S3',
    'And which ONE of these brands is your main provider?': 'Q2',
    'And which ONE of these brands is your main provider? (Other (please specify))': 'Q2_97_Oth',
    'Thinking about \'Origin\', how favourable is your overall impression of them?': 'Q3',
    'How likely are you to recommend \'Origin\' to friends or family?': 'Q4a',
    'You said you would be [unlikely/likely] to recommend \'Origin\'. Why do you say that?': 'Q4b',
    'How would you rate \'Origin\' on each of the following? (Trustworthiness)': 'Q5_1',
    'How would you rate \'Origin\' on each of the following? (Value for money)': 'Q5_2',
    'How would you rate \'Origin\' on each of the following? (Customer service)': 'Q5_3',
    'How would you rate \'Origin\' on each of the following? (Innovation)': 'Q5_4',
    'In the past 12 months, have you seen or heard any advertising for \'Origin\'?': 'Q6',
    'Where did you see or hear advertising for \'Origin\'? (Other (please specify))': 'Q7_97_Oth',
    'Which of the following best describes your current work status?': 'D1',
    'Which of the following best describes your current work status? (Other (please specify))': 'D1_97_Oth',
    'Which of the following best describes your total annual household income?': 'D2',
    'Which of the following best describes your household structure?': 'D3',
    'Which of the following best describes your household structure? (Other (please specify))': 'D3_97_Oth',
    'CompletedDate': 'CompletedDate'
}

# Add special mapping for Q1_97_Oth
q1_other_col = 'Which of the following brands of electricity providers are you aware of? (Other (please specify))'
if q1_other_col in df.columns:
    column_mapping[q1_other_col] = 'Q1_97_Oth'

# Apply renaming
df = df.rename(columns=column_mapping)

# Define column order
column_order = [
    'ID', 'S1', 'S2', 'S3',
    # Q1 multi-response columns
    'Q1_1', 'Q1_2', 'Q1_3', 'Q1_4', 'Q1_5', 'Q1_6', 'Q1_99', 'Q1_97', 'Q1_97_Oth',
    # Q2
    'Q2', 'Q2_97_Oth',
    # Q3-Q5
    'Q3', 'Q4a', 'Q4b', 'Q5_1', 'Q5_2', 'Q5_3', 'Q5_4',
    # Q6
    'Q6',
    # Q7 multi-response columns
    'Q7_1', 'Q7_2', 'Q7_3', 'Q7_4', 'Q7_5', 'Q7_97', 'Q7_97_Oth',
    # Demographics
    'D1', 'D1_97_Oth', 'D2', 'D3', 'D3_97_Oth',
    # Wave and Completed date
    'Wave', 'CompletedDate'
]

# Ensure all columns exist
existing_columns = [col for col in column_order if col in df.columns]
extra_columns = [col for col in df.columns if col not in column_order]

# Reorder columns
df = df[existing_columns + extra_columns]

# Remove original multi-response columns（one-hot already existed）
columns_to_drop = [
    'Which of the following brands of electricity providers are you aware of?',
    'Where did you see or hear advertising for \'Origin\'?'
]

# Drop only main columns, keep other specified ones
df = df.drop(columns=[col for col in columns_to_drop if col in df.columns])



### Step 4.5 – Create “Wave” Variable

A new constructed variable, **Wave**, is created based on the `CompletedDate` field.  

This groups survey responses into weekly periods (Monday–Sunday),  
allowing analysts to monitor changes in brand awareness or perceptions across fieldwork weeks.

**Logic applied:**
1. Convert `CompletedDate` from string to datetime format.  
2. Define the **first week start date** as *Monday, 4th August 2025*.  
3. Calculate each respondent’s **wave number** using:
   \[
   \text{Wave} = \left\lfloor \frac{(\text{date} - \text{start date})}{7} \right\rfloor + 1
   \]
4. Reconvert `CompletedDate` back to string format for export compatibility.

**Wave coding:**
| Code | Week Commencing | Date Range |
|------|------------------|-------------|
| 1 | 4th August | 4–10 Aug |
| 2 | 11th August | 11–17 Aug |
| 3 | 18th August | 18–24 Aug |
| 4 | 25th August | 25–31 Aug |

This derived variable ensures that data can be analysed and reported **by collection wave**,  
which is commonly required in longitudinal and tracking studies.

In [42]:

"""
Create Wave variable and categorize by week
"""

# Convert date format
df['CompletedDate'] = pd.to_datetime(df['CompletedDate'])

# Calculate weekly grouping (Mon–Sun)
def get_week_number(date):
    # Find the most recent Monday (e.g., 4 Aug)
    start_date = datetime(2025, 8, 4)  
    delta = (date - start_date).days
    week_num = (delta // 7) + 1
    return max(1, week_num)  # Ensure at least 1

df['Wave'] = df['CompletedDate'].apply(get_week_number)

# Convert CompletedDate back to string to preserve original format
df['CompletedDate'] = df['CompletedDate'].dt.strftime('%Y-%m-%d %H:%M:%S')



### Step 4.6 – Create Variable and Value Labels

To make the dataset SPSS-ready, variable and value labels are created based on the Example QNA.  
This step ensures that when the `.sav` file is opened in SPSS, each variable and code is properly described.

**Main actions**
1. **Create variable labels** – Assign descriptive question texts to each variable  
   (e.g., `Q3` → “Thinking about 'Origin', how favourable is your overall impression of them?”).  
2. **Add option text for multi-response variables** –  
   For example:  
   - `Q1_1` → “Which of the following brands… – Synergy”  
   - `Q5_3` → “How would you rate 'Origin'… – Customer service”  
3. **Define value labels (codeframes)** – Map numeric codes to their answer text  
   (e.g., `1 = Male`, `99 = Prefer not to say`).  
4. **Add binary value labels** – For multi-response columns (`Q1_x`, `Q7_x`),  
   assign `0 = Not selected`, `1 = Selected`.

**Example:**
| Variable | Variable Label | Value Labels |
|-----------|----------------|---------------|
| `S1` | What is your gender? | 1 = Male, 2 = Female, 3 = Non-binary/Other, 99 = Prefer not to say |
| `Q1_4` | Which of the following brands… – Origin | 0 = Not selected, 1 = Selected |
| `Q5_3` | How would you rate 'Origin'… – Customer service | 1 = Very poor → 5 = Excellent |
| `Q6` | In the past 12 months… | 1 = Yes, 2 = No, 98 = Don’t know |

Once defined, these labels are attached to each variable during the `.sav` export step.


In [43]:

"""
Create variable labels and value labels
"""

# Simple Question texts
question_texts = {
    'S1': 'What is your gender?',
    'S2': 'What is your age?',
    'S3': 'What is your postcode?',
    'Q2': 'And which ONE of these brands is your main provider?',
    'Q3': 'Thinking about \'Origin\', how favourable is your overall impression of them?',
    'Q4a': 'How likely are you to recommend \'Origin\' to friends or family?',
    'Q4b': 'You said you would be [unlikely/likely] to recommend \'Origin\'. Why do you say that?',
    'Q6': 'In the past 12 months, have you seen or heard any advertising for \'Origin\'?',
    'D1': 'Which of the following best describes your current work status?',
    'D2': 'Which of the following best describes your total annual household income?',
    'D3': 'Which of the following best describes your household structure?',
    'Wave': 'Data collection wave',
    'CompletedDate': 'Completion date and time'
}

# Q1 Multi-response option text
q1_option_texts = {
    '1': 'Synergy',
    '2': 'Western Power',
    '3': 'AGL',
    '4': 'Origin',
    '5': 'Horizon Power',
    '6': 'Red Energy',
    '97': 'Other (please specify)',
    '99': 'None of these'
}

# Q5 Multi-response option text
q5_option_texts = {
    '1': 'Trustworthiness',
    '2': 'Value for money',
    '3': 'Customer service',
    '4': 'Innovation'
}

# Q7 Multi-response option text
q7_option_texts = {
    '1': 'TV',
    '2': 'Online / Social media',
    '3': 'Outdoor (billboards, bus stops, etc.)',
    '4': 'Radio',
    '5': 'Print (newspaper, magazine)',
    '97': 'Other (please specify)'
}

# Create variable labels
variable_labels = {
    'ID': 'Respondent ID'
}

# Add single-response question labels
for var, text in question_texts.items():
    variable_labels[var] = text

# Add Q1 multi-response labels
q1_base_text = 'Which of the following brands of electricity providers are you aware of?'
for code, option_text in q1_option_texts.items():
    variable_labels[f'Q1_{code}'] = f"{q1_base_text} - {option_text}"

# Add Q5 multi-response labels
q5_base_text = 'How would you rate \'Origin\' on each of the following?'
for code, option_text in q5_option_texts.items():
    variable_labels[f'Q5_{code}'] = f"{q5_base_text} - {option_text}"

# Add Q7 multi-response labels
q7_base_text = 'Where did you see or hear advertising for \'Origin\'?'
for code, option_text in q7_option_texts.items():
    variable_labels[f'Q7_{code}'] = f"{q7_base_text} - {option_text}"

# Add labels for other text columns
variable_labels.update({
    'Q1_97_Oth': 'Which of the following brands of electricity providers are you aware of? - Other (please specify)',
    'Q2_97_Oth': 'And which ONE of these brands is your main provider? - Other (please specify)',
    'Q7_97_Oth': 'Where did you see or hear advertising for \'Origin\'? - Other (please specify)',
    'D1_97_Oth': 'Which of the following best describes your current work status? - Other (please specify)',
    'D3_97_Oth': 'Which of the following best describes your household structure? - Other (please specify)'
})

# Value labels
value_labels = {
    'S1': {1: 'Male', 2: 'Female', 3: 'Non-binary / Other', 99: 'Prefer not to say'},
    'S2': {2: '18-24', 3: '25-34', 4: '35-44', 5: '45-54', 6: '55-64', 7: '65+'},
    'Q2': {1: 'Synergy', 2: 'Western Power', 3: 'AGL', 4: 'Origin', 5: 'Horizon Power',
            6: 'Red Energy', 97: 'Other (please specify)', 99: 'None of these'},
    'Q3': {1: 'Very unfavourable', 2: 'Somewhat unfavourable', 3: 'Neutral',
            4: 'Somewhat favourable', 5: 'Very favourable'},
    'Q4a': {0: 'Not at all likely', 1: '1', 2: '2', 3: '3', 4: '4', 5: '5',
            6: '6', 7: '7', 8: '8', 9: '9', 10: 'Extremely likely'},
    'Q5_1': {1: 'Very poor', 2: 'Poor', 3: 'Fair', 4: 'Good', 5: 'Excellent', 98: 'Don\'t know'},
    'Q5_2': {1: 'Very poor', 2: 'Poor', 3: 'Fair', 4: 'Good', 5: 'Excellent', 98: 'Don\'t know'},
    'Q5_3': {1: 'Very poor', 2: 'Poor', 3: 'Fair', 4: 'Good', 5: 'Excellent', 98: 'Don\'t know'},
    'Q5_4': {1: 'Very poor', 2: 'Poor', 3: 'Fair', 4: 'Good', 5: 'Excellent', 98: 'Don\'t know'},
    'Q6': {1: 'Yes', 2: 'No', 98: 'Don\'t know'},
    'D1': {1: 'Working full time', 2: 'Working part time', 3: 'Self-employed',
            4: 'Student', 5: 'Unemployed and looking for work', 6: 'Retired', 97: 'Other (please specify)'},
    'D2': {1: 'Less than $30,000', 2: '$30,000–$59,999', 3: '$60,000–$89,999',
            4: '$90,000–$119,999', 5: '$120,000–$149,999', 6: '$150,000 or more', 99: 'Prefer not to say'},
    'D3': {1: 'Live alone', 2: 'Single, no children', 3: 'Single parent with children at home',
            4: 'Couple, no children', 5: 'Couple, with children at home', 6: 'Group household / share house',
            97: 'Other (please specify)'},
    'Wave': {1: 'Week commencing 4th August', 2: 'Week commencing 11th August',
                3: 'Week commencing 18th August', 4: 'Week commencing 25th August'}
}

# # Add binaroy value labels for multi-response variables
for i in range(1, 8):
    if i == 7:  # Q7 only has 5 options
        break
    value_labels[f'Q1_{i}'] = {0: 'Not selected', 1: 'Selected'}

value_labels['Q1_97'] = {0: 'Not selected', 1: 'Selected'}
value_labels['Q1_99'] = {0: 'Not selected', 1: 'Selected'}

for i in range(1, 6):
    value_labels[f'Q7_{i}'] = {0: 'Not selected', 1: 'Selected'}
value_labels['Q7_97'] = {0: 'Not selected', 1: 'Selected'}


## Step 5 – Export to SPSS (.sav)

In this final step, the cleaned dataset is exported to SPSS format (`.sav`) using **pyreadstat**.  
This process preserves all variable labels, value labels, and correct measurement levels  
so that the file is immediately analysis-ready when opened in SPSS or Q.

**Main actions**
1. **Handle missing text values** – Replace `NaN` in open-ended and “Other (please specify)” fields with blank strings to prevent export errors.  
2. **Set data types** – Ensure all numeric variables are integers (`Int64`) and text fields are strings.  
3. **Define measurement levels** – Assign:
   - *Scale* variables → continuous (e.g., `Q4a`, `S3`)  
   - *Ordinal* variables → ordered (e.g., `Q3`, `Q5_1–Q5_4`, `Wave`)  
   - *Nominal* variables → categorical (e.g., demographics, awareness questions)  
4. **Set display widths** – Adjust text column widths (e.g., “Other” responses, `Q4b`) to avoid truncation in SPSS.

**Output files**
- **`cleaned_data.sav`** → Final SPSS-ready file containing all cleaned variables, labels, and formats.  

After this step, the dataset can be directly imported into SPSS or Q for analysis without any further setup.


In [44]:

"""
Save as SPSS format
"""

# Handle empty values in text columns – replace NaN with empty strings
text_columns = [col for col in df.columns if 'Oth' in col or col == 'Q4b' or col == 'CompletedDate']
for col in text_columns:
    if col in df.columns:
        df[col] = df[col].fillna('')

# Set data types
for col in df.columns:
    if col in text_columns:
        # Text columns
        df[col] = df[col].astype(str)
    else:
        # Numeric columns – ensure integer type
        df[col] = pd.to_numeric(df[col], errors='coerce')
        # Convert numeric columns to integers (if possible)
        if col not in ['S3', 'Q4a']:  
            df[col] = df[col].astype('Int64') 

# USe pyreadstat to save .sav file，with clearly defined formats
pyreadstat.write_sav(
    df,
    output_sav_path,
    column_labels=variable_labels,
    variable_value_labels=value_labels,
    variable_measure={
        'scale': ['Q4a', 'S3'],  # Continuous variables
        'ordinal': ['Q3', 'Q5_1', 'Q5_2', 'Q5_3', 'Q5_4', 'Wave'],  # Ordinal variables
        'nominal': [col for col in df.columns if
                    col not in ['Q4a', 'S3', 'Q3', 'Q5_1', 'Q5_2', 'Q5_3', 'Q5_4', 'Wave', 'CompletedDate']]# Nominal variables
    },
    # Explicitly define variable formats and display widths
    variable_display_width={
        'CompletedDate': 20,
        'Q4b': 200,
        'Q1_97_Oth': 100,
        'Q2_97_Oth': 100,
        'Q7_97_Oth': 100,
        'D1_97_Oth': 100,
        'D3_97_Oth': 100
    }
)


# Export to SPSS file (.sav)
print(f"✅ SAV file successfully saved: {output_sav_path}")


✅ SAV file successfully saved: /Users/sunfangyu/Desktop/Charles Data Task/Task1/cleaned_data.sav


### Step 6 – Display the Ratio of Origin vs. Non-Origin Users

This quick check calculates how many respondents identified **Origin** as their main electricity provider (`Q2 = 4`)  
versus those who chose another brand or none.

**Purpose:**
- To perform a quick **sanity check** on the cleaned data (confirming the coding for Q2 works correctly).  
- To provide an **initial insight** into the sample composition — how many are Origin customers compared to non-customers.  
- This proportion can later guide further analysis or segmentation (e.g., comparing brand perceptions between Origin users and others).


In [45]:
# Display the ratio of Origin users vs. non-Origin users
if 'Q2' in df.columns:
    origin_users = (df['Q2'] == 4).sum()
    non_origin_users = len(df) - origin_users
    print(f"Number of Origin users: {origin_users}")
    print(f"Number of non-Origin users: {non_origin_users}")

Number of Origin users: 115
Number of non-Origin users: 78


### Step 7 – Inspect Open-Ended Text Columns

This step performs a final validation check on all **open-ended text fields**, including  
"Other (please specify)" responses and `Q4b` (open comment for recommendation likelihood).

**Purpose:**
- To confirm that all text responses have been **preserved correctly** after cleaning and transformation.  
- To check for **non-empty response counts**, ensuring no data was lost during processing or file export.


In [46]:
# Display inspection results for all text columns
text_columns = [col for col in df.columns if 'Oth' in col or col == 'Q4b']
print(f"\nText column check:")
for col in text_columns:
    if col in df.columns:
        non_empty_count = (df[col] != '').sum()
        print(f"  {col}: {non_empty_count} non-empty values")


Text column check:
  Q1_97_Oth: 23 non-empty values
  Q2_97_Oth: 7 non-empty values
  Q4b: 116 non-empty values
  Q7_97_Oth: 4 non-empty values
  D1_97_Oth: 8 non-empty values
  D3_97_Oth: 3 non-empty values


### Step 8 – Save Processed Excel File for Manual Inspection

After all automated cleaning and formatting steps are completed,  
a final Excel version of the cleaned dataset is exported for manual review.

**Purpose:**
- To provide a **human-readable file** for quick verification of variable order, naming, and coding accuracy.  
- To allow manual spot-checking of skip logic, open-ended responses, and other transformations.  
- This Excel output serves as a QA checkpoint before submission or further analysis.

**Output file:**
- **Filename:** `cleaned_data_check.xlsx`  


In [47]:
# Save processed Excel file for manual inspection
df.to_excel(output_check_path, index=False)
print(f"✅ Excel check file saved: {output_check_path}")

✅ Excel check file saved: /Users/sunfangyu/Desktop/Charles Data Task/Task1/cleaned_data_check.xlsx


---

##  Task 1 Summary – Data Cleaning & Formatting

The dataset has been fully cleaned and formatted according to the provided task instructions.  
All required transformations and checks have been implemented as outlined below:

**Actions Completed:**

1. **Identify and remove invalid records**   

2. **Transform multi-response questions**  

3. **Convert text labels to numeric codes**   

4. **Rename variables according to questionnaire structure**  

5. **Create new variable: “Wave”**  

6. **Define SPSS metadata**  
 
7. **Export final deliverables**  

**Result:**  
✅ The final dataset is fully validated, structured, and ready for import to Q or SPSS for analysis.